In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
# source: https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset
df = pd.read_csv("heart.csv")
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
X = np.array(df.loc[:, df.columns != "output"])
y = np.array(df["output"])

print(f"X: {X.shape}, y: {y.shape}")

X: (303, 13), y: (303,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123
)

In [6]:
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

# %% network class

In [30]:
class NeuralNetworkFromScratch:
    def __init__(self, LR, X_train, y_train, X_test, y_test):
        self.w = np.random.randn(X_train.shape[1])
        self.b = np.random.randn()
        self.LR = LR
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.L_train = []
        self.L_test = []

    def activation(self, x):
        # sigmoid
        return 1 / (1 + np.exp(-x))

    def dactivation(self, x):
        # derivative of sigmoid
        return self.activation(x) * (1 - self.activation(x))

    def forward(self, X):
        hidden_1 = np.dot(X, self.w) + self.b
        activate_1 = self.activation(hidden_1)
        return activate_1

    def backward(self, X, y_true):
        # calc gradients
        hidden_1 = np.dot(X, self.w) + self.b
        y_pred = self.forward(X)
        dL_dpred = 2 * (y_pred - y_true)
        dpred_dhidden1 = self.dactivation(hidden_1)
        dhidden1_db = 1
        dhidden1_dw = X

        dL_db = dL_dpred * dpred_dhidden1 * dhidden1_db
        dL_dw = dL_dpred * dpred_dhidden1 * dhidden1_dw
        return dL_db, dL_dw

    def optimizer(self, dL_db, dL_dw):
        self.b = self.b - self.LR * dL_db
        self.w = self.w - self.LR * dL_dw

    def train(self, iterations):
        for i in range(iterations):
            # random position
            random_pos = np.random.randint(len(self.X_train))
            # forward pass
            y_train_true = self.y_train[random_pos]
            y_train_pred = self.forward(
                self.X_train[random_pos],
            )
            # calculate loss
            L = np.sum(np.square(y_train_pred - y - y_train_true))

            self.L_train.append(L)

            # calculate gradients
            dL_db, dL_dw = self.backward(self.X_train[random_pos], y_train[random_pos])

            # apply gradients
            self.optimizer(dL_db, dL_dw)

            # calc error for test data
            L_sum = 0
            for j in range(len(self.X_test)):
                y_true = self.y_test[j]
                y_pred = self.forward(self.X_test[j])
                L_sum += np.square(y_pred - y_true)

            self.L_test.append(L_sum)

        return "trainning successfully"

In [33]:
# X_train = np.random.randn(200).reshape(100, 2)
# X_train_test = np.random.randn(20).reshape(10, 2)

# Y_train = np.random.randn(100).reshape(100, 1)
# Y_test = np.random.randn(10).reshape(10, 1)

LR = 0.1
iterations = 1000


model = NeuralNetworkFromScratch(
    LR=LR,
    X_train=X_train_scale,
    y_train=y_train,
    X_test=X_test_scale,
    y_test=y_test,
)

model.train(iterations)

'trainning successfully'

In [34]:
model.L_test

[12.496834518253433,
 12.497034717871657,
 12.497480084717655,
 12.497497276127058,
 12.545460216611126,
 12.559866475643895,
 12.560306289719888,
 12.58671000394994,
 12.56267673688379,
 12.562677038300214,
 12.562677034060366,
 12.44431804589774,
 12.44612152198272,
 12.44611782893318,
 12.433231218935807,
 12.4293056696998,
 12.429250304802286,
 12.429245918930668,
 12.466238908844248,
 12.431073625479478,
 12.431073691036492,
 12.431071106500191,
 12.398969187159521,
 12.398966456135701,
 12.40527130002282,
 12.404300743910902,
 12.404289721179403,
 12.405338284787078,
 12.405624234788993,
 12.411901471348513,
 12.412362806627401,
 12.396894529848502,
 12.396907128533698,
 12.397378965495488,
 12.388067113115346,
 12.388216326824413,
 12.388364905129304,
 12.3862302509282,
 12.382123203608675,
 12.3821232340414,
 12.382123234934683,
 12.382119812782314,
 12.382294810549418,
 12.379367909195524,
 12.379367909257898,
 12.406468532888862,
 12.406881634839777,
 12.431314427689587,
 12.